In [2]:
from __future__ import print_function
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from keras.wrappers.scikit_learn import KerasClassifier


def create_model():
    #create model
    model = Sequential()
    model.add(Dense(N_HIDDEN,input_shape=(RESHAPED,)))
    model.add(Activation('relu'))
    #model.add(Dropout(0.3, input_shape=(RESHAPED,)))
    model.add(Dense(N_HIDDEN))
    model.add(Activation('relu'))
    #model.add(Dropout(0.3, input_shape=(RESHAPED,)))
    #model.add(Dropout(0.3, input_shape=(RESHAPED,)))
    model.add(Dense(NB_CLASSES))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer=OPTIMIZER,metrics=['accuracy'])
    return model

np.random.seed(1671)
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 3
OPTIMIZER = Adam()
N_HIDDEN = 128
VALIDATION_SPLIT=0.3
RESHAPED = 171
dataframe_train = pandas.read_csv("TrainingEdgesToAction.csv", header=None)
dataset_train_v = dataframe_train.values
#dataset_train = shuffle(dataset_train,random_state=0)
dataset_train_f = dataset_train_v[1:,0:172]
#print(dataset_train_f)
dataset_train_n = shuffle(dataset_train_f)
print(dataset_train_n)

X_train = dataset_train_n[0:,1:172].astype(float)
Y_train = dataset_train_n[0:,0]

#Y_train = shuffle(Y_test,random_state=0)

dataframe_test = pandas.read_csv("TestingEdgesToAction.csv", header=None)
dataset_test = dataframe_test.values
dataset_test_ft = dataset_test[1:,0:172]
dataset_test_nt = shuffle(dataset_test_ft)
X_test = dataset_test_nt[0:,1:172].astype(float)
Y_test = dataset_test_nt[0:,0]

#X_test = dataset_test[1:,1:172].astype(float)
#Y_test = dataset_test[1:,0]


X_train = X_train.reshape(10500,RESHAPED)
X_test = X_test.reshape(3150,RESHAPED)
X_train /= 255
X_test /= 255




print(X_train.shape[0],'train samples')
print(X_test.shape[0],'test samples')


encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)
print(dummy_y_train)


encoder = LabelEncoder()
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)
print(dummy_y_test)


model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [20,60,80,128]
epochs = [20,50,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, dummy_y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


C:\Users\hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[['LF' 0 3.5 ..., 0.0 0.0 0.0]
 ['FR' 0 3.5 ..., 0.0 0.0 0.0]
 ['LF' 0 3.5 ..., 0.0 0.0 0.0]
 ..., 
 ['FR' 1 3.5 ..., 0.0 0.0 0.0]
 ['LF' 0 3.5 ..., 0.0 0.0 0.0]
 ['FR' 1 3.5 ..., 0.0 0.0 0.0]]
10500 train samples
3150 test samples
[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 ..., 
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_112 (Dense)            (None, 128)               22016     
_________________________________________________________________
activation_112 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_113 (Activation)  (None, 128)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 128)               22016     
_________________________________________________________________
activation_133 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_134 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_134 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_135 (Dense)            (None, 3)                 387       
_________________________________________________________________
activation_135 (Activation)  (None, 3)                 0         
Total params: 38,915
Trainable params: 38,915
Non-trainable params: 0
_________________________________________________________________
______

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_157 (Dense)            (None, 128)               22016     
_________________________________________________________________
activation_157 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_158 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_158 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_159 (Dense)            (None, 3)                 387       
_________________________________________________________________
activation_159 (Activation)  (None, 3)                 0         
Total params: 38,915
Trainable params: 38,915
Non-trainable params: 0
_________________________________________________________________
______

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_181 (Dense)            (None, 128)               22016     
_________________________________________________________________
activation_181 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_182 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_182 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_183 (Dense)            (None, 3)                 387       
_________________________________________________________________
activation_183 (Activation)  (None, 3)                 0         
Total params: 38,915
Trainable params: 38,915
Non-trainable params: 0
_________________________________________________________________
______

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_205 (Dense)            (None, 128)               22016     
_________________________________________________________________
activation_205 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_206 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_206 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_207 (Dense)            (None, 3)                 387       
_________________________________________________________________
activation_207 (Activation)  (None, 3)                 0         
Total params: 38,915
Trainable params: 38,915
Non-trainable params: 0
_________________________________________________________________
______